In [54]:
import numpy as np
import cmath
import time 

Computations of the Riemann Zeta function on the critical line refer to computations of the form $\zeta(\frac{1}{2} + it)$, where we move along the complex plane denoted with the $it$ compenent while being fixed at $\frac{1}{2}$ on the real number line.  

In [65]:
# naive implementation of the Riemann Zeta Function through the Riemann-Siegel function 

def zetaRS(x: int):
    return 0

In [74]:
complex(real=0.5, imag=3)

(0.5+3j)